# Conversão de Anotações para Formato YOLO
Este notebook carrega os ficheiros `train.json` e `test.json`, converte as anotações de caixas delimitadoras para o formato YOLO, copia as imagens e gera os ficheiros `.txt` correspondentes com as labels.

In [1]:
!pip install pillow tqdm

In [2]:
import json
import os
from PIL import Image
from tqdm import tqdm
import shutil

# Diretórios
TRAIN_JSON_PATH = 'train.json'
TEST_JSON_PATH = 'test.json'
TRAIN_IMG_DIR = 'train_data'
TEST_IMG_DIR = 'test_data'
OUTPUT_DIR = 'chestx_det10_yolo'

## Carregar Anotações
Abrimos os ficheiros `train.json` e `test.json` que contêm as anotações no formato original.

In [3]:
# 1. Carregar dados JSON
with open(TRAIN_JSON_PATH, 'r') as f:
    train_data = json.load(f)
with open(TEST_JSON_PATH, 'r') as f:
    test_data = json.load(f)

## Obter Classes Únicas
Extraímos todas as classes únicas presentes no conjunto de dados e mapeamo-las para IDs numéricos.

In [4]:
# 2. Obter todas as classes únicas
all_classes = set()
for item in train_data + test_data:
    all_classes.update(item['syms'])
all_classes = sorted(list(all_classes))
class_to_id = {cls: idx for idx, cls in enumerate(all_classes)}

## Conversão para Formato YOLO

O modelo YOLO (You Only Look Once) espera que as **bounding boxes** estejam no **formato normalizado**, ou seja, todos os valores são representados entre 0 e 1, relativos ao tamanho da imagem.

A função `convert_to_yolo` transforma as caixas de coordenadas absolutas `[x_min, y_min, x_max, y_max]` — que representam os cantos superior esquerdo e inferior direito da caixa — para o formato esperado por YOLO: `[x_center, y_center, width, height]`.

---

### 🧮 Fórmulas de Conversão

Dado:
- `w`: largura da imagem
- `h`: altura da imagem
- `x_min, y_min, x_max, y_max`: coordenadas absolutas da caixa

Calculamos:

$$
x_{center} = \frac{x_{min} + x_{max}}{2 \cdot w}
$$

$$
y_{center} = \frac{y_{min} + y_{max}}{2 \cdot h}
$$

$$
width = \frac{x_{max} - x_{min}}{w}
$$

$$
height = \frac{y_{max} - y_{min}}{h}
$$

---

### 📝 Formato Final por Linha (em ficheiros `.txt`)

O formato geral é:

```
<class_id> <x_center> <y_center> <width> <height>
```

- **`class_id = 2`**  
  → Índice da classe da anomalia (ex.: se `names[2] = 'Nodule'`, então representa um nódulo).

- **`x_center = 0.3`**  
  → O centro da caixa está a **30% da largura da imagem**.

- **`y_center = 0.4`**  
  → O centro da caixa está a **40% da altura da imagem**.

- **`width = 0.2`**  
  → A largura da caixa ocupa **20% da largura da imagem**.

- **`height = 0.2`**  
  → A altura da caixa ocupa **20% da altura da imagem**.

---

### 🧠 Exemplo prático com imagem de 1000×1000

Se a imagem tem 1000 pixels de largura e 1000 pixels de altura:

- `x_center = 0.3 × 1000 = 300 px`
- `y_center = 0.4 × 1000 = 400 px`
- `width = 0.2 × 1000 = 200 px`
- `height = 0.2 × 1000 = 200 px`

Portanto, a caixa irá de:

- `x = 300 - 100 = 200` até `x = 300 + 100 = 400`
- `y = 400 - 100 = 300` até `y = 400 + 100 = 500`

---

### ✅ Vantagens da Normalização

- Permite treinar modelos em imagens de diferentes tamanhos.
- Evita depender da resolução original na inferência.
- É mais leve e eficiente para deteção em tempo real.

In [5]:
# 3. Função para converter boxes para formato YOLO
def convert_to_yolo(box, img_w, img_h):
    x_min, y_min, x_max, y_max = box
    x_center = (x_min + x_max) / 2 / img_w
    y_center = (y_min + y_max) / 2 / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return x_center, y_center, width, height

## Processar Conjuntos de Dados
Esta função converte anotações, cria diretórios, copia imagens e salva as labels no formato YOLO.

In [6]:
# 4. Função para processar imagens e labels
def process_dataset(data, image_dir, subset):
    image_out_dir = os.path.join(OUTPUT_DIR, 'images', subset)
    label_out_dir = os.path.join(OUTPUT_DIR, 'labels', subset)
    os.makedirs(image_out_dir, exist_ok=True)
    os.makedirs(label_out_dir, exist_ok=True)

    for item in tqdm(data, desc=f'Processing {subset}'):
        img_name = item['file_name']
        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_out_dir, img_name.replace('.png', '.txt'))

        if not os.path.exists(img_path):
            print(f"Imagem não encontrada: {img_path}")
            continue

        try:
            with Image.open(img_path) as img:
                img_w, img_h = img.size
        except:
            continue  # ignorar imagens que não abrem

        lines = []
        for cls, box in zip(item['syms'], item['boxes']):
            cls_id = class_to_id[cls]
            x_center, y_center, width, height = convert_to_yolo(box, img_w, img_h)
            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        with open(label_path, 'w') as f:
            f.write('\n'.join(lines))

        # copiar imagem
        dst_img_path = os.path.join(image_out_dir, img_name)
        try:
            shutil.copyfile(img_path, dst_img_path)
        except Exception as e:
            print(f"Erro ao copiar {img_name}: {e}")

## Aplicar Conversão aos Dados
Chamamos a função para converter e copiar as imagens de treino e validação.

In [7]:
# 5. Processar treino e teste
process_dataset(train_data, TRAIN_IMG_DIR, 'train')
process_dataset(test_data, TEST_IMG_DIR, 'val')

Processing train:   0%|          | 0/3001 [00:00<?, ?it/s]

Processing val: 100%|██████████| 542/542 [00:09<00:00, 58.02it/s]


## Gerar Ficheiro `data.yaml`
O ficheiro `data.yaml` é necessário para treinar com YOLOv8.

In [8]:
# 6. Criar ficheiro data.yaml
yaml_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(f"path: {OUTPUT_DIR}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write(f"nc: {len(all_classes)}\n")
    f.write(f"names: {all_classes}\n")